# RAG: Data Ingestion and Retrieval for complex documents


<img src="arch.png" width=500px>

In [1]:
import time
from typing import List, Optional, Union

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from retriever import (
    create_parent_retriever,
    load_embedding_model,
    load_pdf,
    load_reranker_model,
    retrieve_context,
)
from rich import print

Load PDF documents and create retriever

In [2]:
files = [
    "example_data/nw1.pdf",
    "example_data/nw2.pdf",
      # DocLLM paper
]

docs = load_pdf(files=files)

embedding_model = load_embedding_model()
retriever = create_parent_retriever(docs, embedding_model)
reranker_model = load_reranker_model()

Initialize Langchain's Ollama Chat API integration using `mistral` model and create the chain with prompt template.

In [23]:
llm = ChatOllama(model="orca2", temperature=0)
prompt_template = ChatPromptTemplate.from_template(
    (
        "Please answer the following question based on the provided `context` that follows the question and answer it directly.\n"
        "If you do not know the answer then just say 'I do not know'\n"
        "question: {question}\n"
        "context: ```{context}```\n"
    )
)
chain = prompt_template | llm | StrOutputParser()

Retrieve document and run the chain using `context` and `question`.

In [31]:
# Run through some sample queries
samples = [
    "How many properties available for investment",
    "What is the available rentable area 1401 N Casaloma Dr, Appleton, WI",
    "Summarize the demographics at Appleton location",
    "Which property I should invest from all the properties available o make profit?"
]

In [32]:
for query in samples:
    print(query)
    context = retrieve_context(
        query, retriever=retriever, reranker_model=reranker_model
    )[0]
    output = chain.invoke({"context": context[0].page_content, "question": query})
    print("LLM Response:", output)

    print("\n", "=" * 100, "\n\n")

How many properties available for investment

LLM Response: To answer the question based on the provided context, we need to identify what kind of properties are
available for investment and how many of them are in the Appleton Metro area. We can use the following steps:

- First, we need to understand that the context is a description of two investment opportunities: one is a 
franchise opportunity with Anytime Fitness, which is a fitness gym business, and the other is a NNN (National 
Retail Properties) investment opportunity, which is a net lease property.
- Second, we need to count how many of each type of property are listed in the context. For the franchise 
opportunity, we can see that there are two locations with different addresses: one is 54915 and the other is 54913.
For the NNN investment opportunity, we can see that there is only one location given: Appleton Metro.
- Third, we need to add up the number of properties for each type of investment and compare them with the question,
which asks how many properties are available for investment. We can see that there are two franchise locations and 
one NNN property in total, so the answer is three properties.

### Final answer: Three properties

====================================================================================================

What is the available rentable area 1401 N Casaloma Dr, Appleton, WI

LLM Response: To answer the question based on the context, we need to find the information that relates to the 
rentable area of 1401 N Casaloma Dr, Appleton, WI. We can scan the context for keywords such as "rentable area", 
"square feet" or "SF". We can also look at the property overview section, which usually summarizes the main 
features of a property.

We find that the property overview section has the following line: ±7,609 SF Rentable Area. This means that the 
rentable area of 1401 N Casaloma Dr, Appleton, WI is approximately 7,609 square feet. We can also check the 
building photos section to verify this information, as it might show a sign or a plaque with the rentable area on 
the property.

### Final answer: The available rentable area of 1401 N Casaloma Dr, Appleton, WI is approximately 7,609 square 
feet.

...

LLM Response: To answer the question based on the context, I will follow these steps:

- Identify the relevant section of the context that contains the demographics information. This is the section 
labeled "Demographics 1401 N Casaloma Dr, Appleton, WI".
- Extract the data from the section and organize it into a table or a list. For example:

Population within 1 mile 7,715
Population within 3 miles 83,808
Population within 5 miles 206,245
Population within 10 minutes 112,732
Population within 15 minutes 7,997
Households within 1 mile 3,623
Households within 3 miles 35,362
Households within 5 miles 85,173
Median HH Income within 1 mile $60,050
Median HH Income within 3 miles $61,948
Median HH Income within 5 miles $61,533
Employees within 10 minutes 82,484
Employees within 15 minutes 75,045
Employees within 15 miles 137,747

- Summarize the demographics at Appleton location by using the data from the table or list. For example:

The demographics at Appleton location are mostly concentrated within a 5 mile radius, with a population of 206,245 
and a median household income of $61,533. The area has a high demand for employees, with over 137,000 workers 
living nearby. The nearest employment centers are within 15 minutes drive away.

### Final answer: The demographics at Appleton location are mostly concentrated within a 5 mile radius, with a 
population of 206,245 and a median household income of $61,533.

====================================================================================================

Which property I should invest from all the properties available o make profit?

LLM Response: To answer the question based on the context, we need to do the following steps:

- Identify the relevant information in the context that relates to the question
- Compare and contrast the different properties available for investment
- Evaluate the pros and cons of each property based on their location, market size, demand, competition, and profit
potential
- Choose the best property for investment based on the criteria and preferences of the user

Based on these steps, we can answer the question as follows:

The context provides information about two properties that are available for investment in the Appleton Metro area.
They are:

- Anytime Fitness gyms at 54915 and 54913, which are franchise opportunities run by Bandon Fitness, a large and 
successful fitness company
- NNN Investment Opportunity, which is a single-tenant net lease property with an unknown tenant and location

To compare and contrast these properties, we can consider the following factors:

- Location: The Anytime Fitness gyms are located in different areas of Appleton Metro, while the NNN Investment 
Opportunity does not specify where it is. This means that the Anytime Fitness gyms have more potential to reach a 
wider customer base and generate more revenue, but they also face more competition from other fitness centers and 
gyms in the area. The NNN Investment Opportunity may have less competition, but it also has more uncertainty about 
its tenant and location, which may affect its profitability and value.
- Market size: The Anytime Fitness gyms are part of a global franchise with over three million members and 3,000 
locations worldwide, while the NNN Investment Opportunity is not disclosed. This means that the Anytime Fitness 
gyms have more brand recognition and awareness, which may attract more customers and generate more revenue. The NNN
Investment Opportunity may have a smaller or unknown market size, which may limit its potential and value.
- Demand: The Anytime Fitness gyms cater to the health and fitness needs of people in Appleton Metro and beyond, 
while the NNN Investment Opportunity does not specify what kind of business it is. This means that the Anytime 
Fitness gyms have more demand from a diverse and growing segment of consumers, which may increase their occupancy 
rate and revenue. The NNN Investment Opportunity may have less demand from a specific or niche market, which may 
lower its occupancy rate and revenue.
- Profit potential: The Anytime Fitness gyms are run by Bandon Fitness, a successful and experienced franchisee 
that has annual gross profits of over $80 million and assets of over $110 million, while the NNN Investment 
Opportunity is not disclosed. This means that the Anytime Fitness gyms have more profit potential from their 
high-quality service, brand recognition, and franchise support, which may increase their net income and value. The 
NNN Investment Opportunity may have less profit potential from its unknown tenant, location, and business model, 
which may decrease its net income and value.

Based on these factors, we can evaluate the pros and cons of each property as follows:

- Pros: The Anytime Fitness gyms are part of a global franchise with high market size, demand, and profit 
potential. They have the resources and support to run a successful business that makes a difference in people's 
lives. They offer a convenient and affordable recurring-revenue model for entrepreneurs who seek the opportunity to
be their own boss and achieve a healthy work/life balance.
- Cons: The Anytime Fitness gyms face more competition from other fitness centers and gyms in the area, which may 
affect their occupancy rate and revenue. They also have more uncertainty about their location, which may affect 
their visibility and accessibility to customers.
- Pros: The NNN Investment Opportunity has less competition from its unknown tenant and location, which may 
increase its occupancy rate and revenue. It also has a smalle

====================================================================================================